In [1]:
import librosa
# !pip install moviepy
from moviepy.editor import VideoFileClip

In [2]:
import os

data_list = os.listdir('./data/')

for file in data_list:  
    print(file)
    if file == 'audio_data':
        continue
    video_path = f'data/{file}/raw/movie0/movie-sound.mp4'
    output_path = f'data/audio_data/{file}.wav'
    # video = VideoFileClip(video_path)
    # audio = video.audio
    # audio.write_audiofile(output_path)

    # audio.close()
    # video.close()

audio_data
chord_0
chord_1
chord_2
climb_0
climb_1
climb_2
climb_3
climb_4
climb_5
huanlesong
littlestar
long_octave_0
long_octave_1
long_octave_2
octave_0
octave_1
octave_10
octave_11
octave_2
octave_3
octave_4
octave_5
octave_6
octave_7
octave_8
octave_9


In [3]:
import scipy.signal

audio_path = output_path
print(audio_path)
y, sr = librosa.load(audio_path, sr=44100)
print(sr)
# y = librosa.resample(y, orig_sr=sr, target_sr=4000)
# mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
# print(sr)
# mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=16000, n_mels=128)
# print(mel_spectrogram.shape)

duration = librosa.get_duration(y=y, sr=sr)
print(duration)

target_sr = 2000
num_samples = int(duration * target_sr)
print('num_samples:', num_samples)

resampled_y = scipy.signal.resample(y, num_samples)
# resampled_y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
print(resampled_y.shape)

mel_spectrogram = librosa.feature.melspectrogram(y=resampled_y, sr=4000, n_mels=128, hop_length=1)[:,:-1]
print('mel_spectrogram shape: (f_in, frame_num)', mel_spectrogram.shape)

data/audio_data/octave_9.wav
44100
7.6
num_samples: 15200
(15200,)
mel_spectrogram shape: (f_in, frame_num) (128, 15200)


In [4]:
import torch.nn.functional as F

def linear_interpolation(features, input_fps, output_fps, output_len=None):
    features = features.transpose(1, 2)
    seq_len = features.shape[2] / float(input_fps)
    if output_len is None:
        output_len = int(seq_len * output_fps)
    output_features = F.interpolate(features,size=output_len,align_corners=True,mode='linear')
    return output_features.transpose(1, 2)

from transformers import Wav2Vec2Processor, Wav2Vec2Model

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
y_16000, sr = librosa.load(audio_path, sr=16000)
input_values = processor(y_16000, return_tensors="pt").input_values
print(y_16000.shape)
print(input_values.shape)

audio_model = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base-960h')
audio_model.feature_extractor._freeze_parameters()

hs = audio_model.feature_extractor(input_values)
hs = linear_interpolation(hs, 50, 30,output_len=mel_spectrogram.shape[1])
print(hs.shape)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


(335160,)
torch.Size([1, 335160])


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 15200, 1047])


In [5]:
from model.layer import BasicBlock
import torch.nn as nn
import torch

class WavEncoder(nn.Module):
    def __init__(self, out_dim, audio_in=1):
        super().__init__() 
        self.out_dim = out_dim
        self.feat_extractor = nn.Sequential( 
                BasicBlock(audio_in, out_dim//4, 15, 5, first_dilation=1600, downsample=True),
                BasicBlock(out_dim//4, out_dim//4, 15, 6, first_dilation=0, downsample=True),
                BasicBlock(out_dim//4, out_dim//4, 15, 1, first_dilation=7, ),
                BasicBlock(out_dim//4, out_dim//2, 15, 6, first_dilation=0, downsample=True),
                BasicBlock(out_dim//2, out_dim//2, 15, 1, first_dilation=7),
                BasicBlock(out_dim//2, out_dim, 15, 3,  first_dilation=0,downsample=True),     
            )
    def forward(self, wav_data):
        # print(wav_data.shape)   
        if wav_data.dim() == 2:
            wav_data = wav_data.unsqueeze(1) 
        else:
            wav_data = wav_data.transpose(1, 2)
        out = self.feat_extractor(wav_data)
        return out.transpose(1, 2)

In [6]:
y_beat, sr_beat = librosa.load(audio_path, sr=16000)
y_beat = librosa.feature.melspectrogram(y=y_beat, sr=16000, n_mels=128, hop_length=1)
print(y_beat.shape)
y_beat = y_beat.transpose(1, 0)
print(y_beat.shape)
# y_beat = torch.tensor(y_beat).unsqueeze(0)
wav_encoder = WavEncoder(256, 1).cuda()
y_beat = wav_encoder(torch.tensor(y_beat).cuda())


(128, 121601)
(121601, 128)


OutOfMemoryError: CUDA out of memory. Tried to allocate 19.22 GiB (GPU 0; 24.00 GiB total capacity; 19.29 GiB already allocated; 2.46 GiB free; 19.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF